In [1]:
!pip install -U -q datasets
!pip install -U -q trl
!pip install -U -q peft
!pip install -U -q bitsandbytes
!pip install -U -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%ls "/content/drive/MyDrive/NLP_COURSE"
%cd "/content/drive/MyDrive/NLP_COURSE"

credentials.py  finetuning_dataset.jsonl  logs/    __pycache__/
finetune.ipynb  Inference.ipynb           output/
/content/drive/MyDrive/NLP_COURSE


In [4]:
PROJECT_ROOT_PATH = "/content/drive/MyDrive/NLP_COURSE"
OUTPUT_DIR = f"{PROJECT_ROOT_PATH}/output"

In [5]:
# @title Dependencies

import sys, os
from pprint import pprint
from pathlib import Path
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()

import json
import numpy as np
import pandas as pd

import datasets as ds
from datasets import load_from_disk
import torch
from trl import SFTTrainer
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
    )

from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    LlamaForCausalLM,
)

from credentials import HF_TOKEN
from huggingface_hub import login
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


# load the base model in 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True,
)
model.config.use_cache = False

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True,
    trust_remote_code=True,
    use_auth_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model = PeftModel.from_pretrained(
    model,
    f"{OUTPUT_DIR}/ft_amazon",
    is_trainable=True,
    repo_type="file"
)
model = model.eval()

In [8]:
def get_text(question):
    text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    You are a helpful assistant for Amazon Shoe department. Your task is to help the customers find the best shoes of the world! <|eot_id|><|start_header_id|>user<|end_header_id|>

    {0} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""".format(question)
    return text


def generate_answer(model,
                    tokenizer,
                    question):

    model = model.eval()

    text = get_text(question)
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    generation_config = {
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
        "max_length":300,
        "num_return_sequences":1,
    }

    with torch.no_grad():
        outputs = model.generate(**inputs,**generation_config)

    decoded_output = tokenizer.decode(outputs[0])

    split_token = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    decoded_output = decoded_output.split(split_token)[-1]
    return decoded_output.replace("<|eot_id|>", "")


In [9]:
output = generate_answer(model,
                         tokenizer,
                         "What are the best women shoes from Geox?")
pprint(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('\n'
 '\n'
 'Hello! Here are some women shoes from Geox that you might find appealing:\n'
 "- Geox Women's D Rubidia F Low-Top Sneakers (£32.15 - £64.51)\n"
 "- Geox Women's D Nebula C Sneaker (£50.00 - £75.00)\n"
 "- Geox Women's D Nebula a Sneaker (£40.00 - £70.00)\n"
 "- Geox Women's D Myria a Sneaker (£25.96 - £59.00)\n"
 "- Geox Women's D Kaula a Sneaker (£34.03 - £82.55)\n"
 "- Geox Women's D Waviness B Low-Top Sneakers (£34.50 - £50.00)\n"
 "- Geox Women's D Kaula B Sneaker (£34.99 - £59.00) ")
